In [ ]:
import arcgis
incident_name = input("Incident Name: ")
agol_username = input("AGOL Username: ")
agol_url = "https://arcgis.com"

gis_object = arcgis.GIS(agol_url, agol_username)

In [ ]:
def create_view_incident(incident,  core_data_item):

    print('Creating Hosted Feature View')

    # Create Hosted Feature View
    no_space_title = incident.replace(' ', '_')
    veg_flc = arcgis.features.FeatureLayerCollection.fromitem(core_data_item)
    hfv_item  = veg_flc.manager.create_view('Incident_Response_{}'.format(no_space_title))

    # Update Hosted Feature View Settings
    
    # create feature layer collection from new item
    hfv_flc = arcgis.features.FeatureLayerCollection.fromitem(hfv_item)
    
    # enable editing, sync, change-tracking
    hfv_flc.manager.update_definition({
        'capabilities': 'Query, Editing, Update, Sync, Create, Update, Delete, ChangeTracking',
    })

    # Apply Definition Filter
    for layer in hfv_flc.layers:
        types = [dict(typer) for typer in list(layer.properties.types)]
        for typer in types:
            typer['templates'][0]['prototype']['attributes']['INCIDENT'] = incident
                    
        layer.manager.update_definition({
            'types':types,
            'viewDefinitionQuery': "INCIDENT = '{}'".format(incident)
        })

    return hfv_item

In [ ]:
def create_nrm_pi_web_map(incident, wm_template, new_hfv_item):
    
    print('Creating Web Map')
    
    # Pull Template Data from original web map
    template_json = wm_template.get_data()
    
    # Swap Operational Layer Values to Match Newly Created HFVs
    template_op_lyrs = template_json['operationalLayers']
    for op_lyr in template_op_lyrs:
        # Inspection Line - 
        # change filter in web map
        if op_lyr['title'] == 'Inspection Line':
            op_lyr['layerDefinition']['definitionExpression']="PROJ = '{}'".format(incident)
            
        # Vegetation Point
        # switch layer to new hosted feature view
        elif op_lyr['title'] == 'Vegetation Point':
            op_lyr['url']   = '{}/0'.format(new_hfv_item.url)
            op_lyr['itemId'] = new_hfv_item.id  
            
        # incident boundary
        # change filter in web map
        elif op_lyr['title'] == 'Incident Perimeter':
            op_lyr['layerDefinition']['definitionExpression']= "INCDENTNAME = '{}'".format(incident)
            
        # Unexpected Layers Are Effectively Passed
        else:
            print('Operational Layer Has No Available Swap: {}'.format(op_lyr['title']))

    # Create Basic Map
    map_item = arcgis.mapping.WebMap().save({
        'title': "Rapid Response - "+incident,
        'snippet': 'Rapid Response',
        'tags': ["Rapid Response"],
        'typeKeywords': [
            'ArcGIS Online',
            'Collector',
            'Data Editing',
            'Explorer Web Map',
            'Map',
            'Offline',
            'Online Map',
            'Web Map'
        ]
    })
    
    # Configure the Basic Map
    
    map_item.update({
        "text" : template_json
    })

    return map_item
